In [1]:
import pandas as pd
import json
import requests
import random
from lxml import html
import re
import time

from  util.spider import Spider
from util.http_utility import get_http_headers
from util.user_agent import get_random_ua

# Scrape Japan - Delishtv



https://delishkitchen.tv/

In [2]:
class DelishKitchen(Spider): #inherit Spider Class
    """This class extends the main Spider Class for Scraping Delish TV recipe website.

    Args:
        Spider (_type_): _description_
    """

    def __init__(self, main_page, seeds, listing, attrs, header=None):
        super().__init__(main_page,seeds,listing,attrs, header)

    #Process the duration format from the recipes website
    def process_time(self,time_str):
        if 'S' in time_str:
            #hours = time_str.split("H")[0]
            seconds  = time_str.split("PT")[1].split('S')[0]
            #minutes = re.findall(r'\b\d+\b', time_str.split("H")[1])
            # print(hours)
            # print(minutes)
            return seconds
        else:
            return time_str
    
    # def check_normalize_space(self,xpath):
    #     super().check_normalize_space(xpath)

    def scrape_one_item(self,url):
        """To scrape one item from a given url.

        Args:
            url (str): url of the item

        Returns:
            dict: dict of the item
        """
        session = requests.Session() 
        session.headers.update(self.header)

        #add home page url to the obtained item url if item url is not complete
        if self.main_page not in url:
            res = session.get(self.main_page + url)
        else:
            res = session.get(url)

        if res.status_code == 200:
            try:
                #print(res.status_code)
                doc = html.document_fromstring(res.text)
                #set default values for variables
                name, total_time, ingredients, instructions, servings, category, prep_time, cook_time = '','','','','','','',''

                doc = html.document_fromstring(res.text)
                
                #get data from script application json with recipeIngredient
                script_data = json.loads(doc.xpath("normalize-space(//script[@type='application/ld+json'][contains(text(), 'recipeIngredient')])"))
                #print(script_data)

                name = script_data.get('name')

                ingredients = script_data.get('recipeIngredient')
                ingredients = ' '.join(ingredients)# join ingredients

                instructions = []
                inst_list_web = script_data.get('recipeInstructions')

                for instruction in inst_list_web:
                    instructions.append(instruction['text'])
                instructions= ' '.join(instructions) #join all as one


                servings = script_data.get('recipeYield')

                category = doc.xpath('normalize-space(//ul[@class="categories"])')


                total_time = script_data.get('totalTime')
                total_time = self.process_time(total_time)
                
                #for this website the cooktime and preptime are misleading
        
                #cook_time = script_data.get('cookTime')
                #prep_time = script_data.get('prepTime')

                return {'name': name, 'total_time': total_time, 'ingredients': ingredients, 'instructions': instructions, 'servings': servings,
                'category': category, 'prep_time': prep_time, 'cook_time': cook_time,}
            except Exception as e:
                raise Exception('something is wrong for item: {}'.format(url)) from e


        else:
            raise Exception('Cannot open one menu url, status code = {}'.format(res.status_code))
    

    def get_items_from_page(self, page_url):
        """Get all item urls from the current page listing

        Args:
            page_url (str): url of the page

        Returns:
           list: list containing the url of items in the current page
        """
        session = requests.Session() 
        session.headers.update(self.header)
        res = session.get(page_url)
        if res.status_code == 200:
            doc = html.document_fromstring(res.text)
            urls = doc.xpath(self.listing['items'])
            print('got items from url: ', page_url)
            return urls
        else:
            raise Exception('Cannot open the menu url, status code = {}'.format(res.status_code))


In [3]:
listing={'items': '//ul[@class="columns with-max-three"]//div[contains(@class,"delish-renewal-recipe-item-card")]//a/@href', 'next': { 'next_page_str': '?page={}', 'type': 'url'}}
seeds = ['https://delishkitchen.tv/categories/19878', 'https://delishkitchen.tv/categories/655', 'https://delishkitchen.tv/categories/17359', ]


In [4]:
delishtv_spider= DelishKitchen('https://delishkitchen.tv', seeds= seeds, listing =listing,attrs= '')

In [5]:
delishtv_spider.scrape_one_item('https://delishkitchen.tv/recipes/216142397518644383')

{'name': '冷凍餃子を使ってお手軽に♪ ニラもやしの餃子鍋',
 'total_time': '1200',
 'ingredients': '冷凍餃子 12個 豆もやし 1袋 ニラ 1/2束 しめじ 1パック 酒 大さじ1 塩 小さじ1/3 鶏ガラスープの素 小さじ1/2 おろしにんにく 小さじ1 おろししょうが 小さじ1 水 300cc 唐辛子(輪切り) 少々 ポン酢しょうゆ 大さじ2 ラー油 適量',
 'instructions': 'ニラは3cm幅に切る。しめじは根元を切り落とし、小房に分ける。 ボウルに豆もやし、ニラ、しめじを入れて混ぜる。 別のボウルに☆を入れて混ぜる(鍋スープ)。 鍋に半量の2を敷き詰め、餃子は鍋に沿うように並べ入れて、中央に残りの2をのせる。鍋スープをまわしかけ、中火に熱し、煮立ったらふたをして5分程煮る。 4にポン酢しょうゆ、ラー油を添える。',
 'servings': '2人分',
 'category': 'その他の鍋料理',
 'prep_time': '',
 'cook_time': ''}

In [6]:
result_list = delishtv_spider.start_scrape(max_pages=1) #max page is only 130

spider is scraping page: 1
spider is scraping url:  https://delishkitchen.tv/categories/19878?page=1
got items from url:  https://delishkitchen.tv/categories/19878?page=1
scraping - multithreaded, n items:  30


In [7]:
result_list

[{'name': 'レンジでできる！ 豚バラもやしのバターポン酢',
  'total_time': '600',
  'ingredients': '豚バラ薄切り肉 150g もやし 1袋(200g) 酒 大さじ1 有塩バター 大さじ1 ポン酢しょうゆ 大さじ2 かつお節 1パック 七味唐辛子 適量 細ねぎ（刻み） 適量',
  'instructions': '豚肉は食べやすい大きさに切る。耐熱容器にもやしを入れ、かつお節を振り入れ、その上に切った豚肉を広げてのせる 酒を全体に振りかけてバターをところどころに乗せ、ふんわりラップをして600Wのレンジで5～6分加熱する。 豚肉に火が通ったらポン酢しょうゆを全体を混ぜる。器に盛り、七味唐辛子をふって刻んだ細ねぎを乗せて完成！',
  'servings': '2人分',
  'category': 'その他の野菜料理豚肉豚バラ薄切り肉よく使う野菜',
  'prep_time': '',
  'cook_time': ''},
 {'name': '甘い味噌味が食欲そそる！ 豚じゃが味噌炒め',
  'total_time': '900',
  'ingredients': 'じゃがいも 1個 豚こま切れ肉 70g ごま油 大さじ1/2 ネギ 適量 白ごま 適量 酒 小さじ1 片栗粉 小さじ1 味噌 小さじ1 酒 小さじ1 砂糖 小さじ2 みりん 小さじ1',
  'instructions': '皮ごとよく洗ったじゃがいもを500wのレンジで5分加熱する \n粗熱が取れたら一口大に切る 豚肉に酒、片栗粉を揉み込む フライパンにごま油を熱し、肉を焼いていく \n焼き色がついてきたらじゃがいもを加える 味噌、砂糖、酒、みりんをよく混ぜて加え、全体にからませる \n仕上げにネギ、ごまを散らして完成！',
  'servings': '1人分',
  'category': '煮物(野菜)肉じゃが秋冬の旬野菜ねぎ',
  'prep_time': '',
  'cook_time': ''},
 {'name': '時短がっつりおかず！ チーズ焼き卵巾着',
  'total_time': '900',
  'ingredients': '油揚げ 1枚 卵 2個 スライスチーズ 1枚 サラダ油 小さじ2 酒 大さじ1 

In [8]:
print(result_list)

[{'name': 'レンジでできる！ 豚バラもやしのバターポン酢', 'total_time': '600', 'ingredients': '豚バラ薄切り肉 150g もやし 1袋(200g) 酒 大さじ1 有塩バター 大さじ1 ポン酢しょうゆ 大さじ2 かつお節 1パック 七味唐辛子 適量 細ねぎ（刻み） 適量', 'instructions': '豚肉は食べやすい大きさに切る。耐熱容器にもやしを入れ、かつお節を振り入れ、その上に切った豚肉を広げてのせる 酒を全体に振りかけてバターをところどころに乗せ、ふんわりラップをして600Wのレンジで5～6分加熱する。 豚肉に火が通ったらポン酢しょうゆを全体を混ぜる。器に盛り、七味唐辛子をふって刻んだ細ねぎを乗せて完成！', 'servings': '2人分', 'category': 'その他の野菜料理豚肉豚バラ薄切り肉よく使う野菜', 'prep_time': '', 'cook_time': ''}, {'name': '甘い味噌味が食欲そそる！ 豚じゃが味噌炒め', 'total_time': '900', 'ingredients': 'じゃがいも 1個 豚こま切れ肉 70g ごま油 大さじ1/2 ネギ 適量 白ごま 適量 酒 小さじ1 片栗粉 小さじ1 味噌 小さじ1 酒 小さじ1 砂糖 小さじ2 みりん 小さじ1', 'instructions': '皮ごとよく洗ったじゃがいもを500wのレンジで5分加熱する \n粗熱が取れたら一口大に切る 豚肉に酒、片栗粉を揉み込む フライパンにごま油を熱し、肉を焼いていく \n焼き色がついてきたらじゃがいもを加える 味噌、砂糖、酒、みりんをよく混ぜて加え、全体にからませる \n仕上げにネギ、ごまを散らして完成！', 'servings': '1人分', 'category': '煮物(野菜)肉じゃが秋冬の旬野菜ねぎ', 'prep_time': '', 'cook_time': ''}, {'name': '時短がっつりおかず！ チーズ焼き卵巾着', 'total_time': '900', 'ingredients': '油揚げ 1枚 卵 2個 スライスチーズ 1枚 サラダ油 小さじ2 酒 大さじ1 みりん 大さじ2 しょうゆ 大さじ2 砂糖 大さじ1', 'inst